# Welcome 

Oh hey.

In this homework, we will go through the classical reinforcement learning method, Q-learning. Then we will also describe an approach to Thompson Sampling for unknown MDPs.

# Q-Learning

For an MDP with a finite number of states and finite number of actions, Q-learning. We initialize $Q_0(s,a)$ arbitrarily for all state-action pairs. Then given samples $(S_t,A_t,R_{t+1},S_{t+1})$ of states, actions, rewards, and next states, we update $Q_t$ by:
$$
Q_{t+1}(S_t,A_t) = Q_t(S_t,A_t) + \alpha_t \left(R_{t+1}+\gamma \max_{a\in\mathcal{A}} Q_t(S_{t+1},a) - Q_t(S_t,A_t)\right)
$$

Here $\gamma \in[0,1]$ is the discount factor, and $\alpha_t\in (0,1)$ are tunable step size parameters.

Q-learning is guarnateed to converge to the optimal action-value function, $Q^\star(s,a)$ under the following conditions:
- $\gamma \in (0,1)$
- $\alpha_t$ satisfies standard hypotheses of stochastic approximation, i.e. $\sum_{t=0}^\infty \alpha_t =\infty $ and $\sum_{t=0}^\infty \alpha_t^2 <\infty$
- Each state-action pair $(s,a)$ is visited infinitely often, it can be shown that $Q_t$ converges to the optimal action-value function. 

Note that the convergence results holds regardless of what policy is used to generate the data. (This is why Q-learning is known as an *off-policy* method.)

Here are a few common ways to set $\alpha_t$:
- Pick a small constant value, $\alpha_t = \alpha\in (0,1)$. For example $\alpha=0.01$. We get no convergence guarantees, but it is simple and often works well.
- Let $\alpha_t = 1/(t+1)$. This satisfies the stochastic approximtion assumptions, and is easy to implement.
- Let $n_t$ be the number of times that the pair $(S_t,A_t)$ has been seen so far, and let $\alpha_t = 1/n_t$. This also satisfies the stochastic approximation assumptions, but is slightly harder to implement.

Of course, there are many ways to attempt to tune the step size. (And it is easy to spend a lot of time doing this.)

# Control from the Q estimate

A natural method to generate actions is to use the estimate, $Q_t$, as though it were the correct optimal action-value function, but then augment the strategy to enforce some exploration. The simplest method to force exploration is the $\epsilon$-greedy method, described below.


Choose $\epsilon\in (0,1)$ to be small.
At time $t$, with probability $1-\epsilon$ choose $A_t$ to maximize $Q_t(S_t,a)$. Otherwise, choose $A_t\in\mathcal{A}$ uniformly at random.

You can sometimes get better performance out of $\epsilon$-greedy by letting it vary with time. Typically, $\epsilon$ would start off high and then slowly converge to $0$.

In [ ]:
# Let's code!
import gym
import numpy as np
import scipy.stats as st
import numpy.random as rnd
import scipy.linalg as la
import  matplotlib.pyplot as plt
%matplotlib inline



# Load the environment. This is the same as last time.
env = gym.make('FrozenLake-v0')
nS = env.observation_space.n
nA = env.action_space.n

# Coding Question 

Code up Q-learning with the $\epsilon$-greedy strategy. Run $10^4$ episodes and let $G_{k}$ be the total undiscounted return of episode $k$:
$$
G_k = \sum_{t=0}^T R_t
$$
(I.e. the sum of all rewards of episode $k$.) 

Make a plot of the running average of the returns: $\frac{1}{k}\sum_{j=1}^kG_j$ versus episode number, $k$.

Hints and Comments:
- The previous homework showed how to simulate episodes of the Frozen Lake environment
- As in the previous homework, discard episodes in which the time limit is reached. 
- Do not reset your Q values at each episode. Keep learning!
- You should only  do the Q-learning update step when the time limit has not been reached. (It won't cause much problem if you do)
- An optimistic initialization, e.g. $Q_0(s,a)=1$ for all state-action pairs, encourages early exploration, and can greatly speed up learning.
- If $Q_0(s,a)\ne 0$, you will need to use a discount factor $\gamma <1$  in the update step. (However, do not use the discount factor when calculating $G_k$)
- If $(S_t,A_t,R_{t+1},S_{t+1})$ is such that $S_{t+1}$ is terminal, you should make sure that $Q_t(S_{t+1},a)=0$ for all actions, $a$.
- The best possible average return value is around $0.82$. 
- You can get very close to the best value, with careful choices of $Q_0(s,a)$, $\alpha_t$, $\epsilon$, and $\gamma$. Try to get it to work well, but don't let it consume your life. (I got fast convergence to near-optimial performance by setting $Q_0(s,a)=1$, $\gamma=0.99$ and $\epsilon=0$. In this case, all of the exploration is due to the optimistic initialization. The performance varies a lot depending on the parameters, and in many cases, it barely learns at all.)

In [ ]:
# Put your code here

# Thompson Sampling for MDPs

Here we describe a simple procedure for doing Thompson Sampling for MDPs. The main advantage over the Q-learning method is that it works well, and does not require careful tuning. The disadvantage is that is a more complex algorithm, both in terms of implementation and in terms of runningn time.
For the particular example, it learns a bit slower than a well-tuned version of Q-learning, but much faster than poorly tuned versions. 

# Outline of the Algorithm

Before getting to the details of the algorithm, we'll describe it at a high level.

* Start with a uniform priors over transition and reward probabilities.
* For each episode do the following:
    - Construct posterior distribution for the transition and reward probabilities given all of the trajectory data observed so far. (This will be described in greater detail below.)
    - Sample this posterior to construct a Markov Decision Process
    - Find an optimal (or near optimal) policy for this MDP. (In my code, I just used one step of policy iteration at the beginning of each episode.)
    - Run an episode using this policy
    
Note that the MDP is sampled before the episode starts and is held  fixed until the end of the episode. 
Now we discuss how we get these distributions.
    
# Beta Distributions and Binary Rewards

In the Frozen Lake example, the rewards are binary. Based on the assumptions of MDPs it follows that $R_{t+1}$ is a Bernoulli random variable when conditioned on $(S_t,A_t)$. The *conjugate prior* of the Bernoulli distributionn is a beta distribution.

For simpler notation in the discussion, we will consider Bernoulli random variables in isolation.

Let $\Theta\in[0,1]$ be a beta random variable with parameters $\alpha >0$ and $\beta > 0$. So, the density of $\Theta$ is given by:
$$
p(\Theta) = \frac{1}{B(\alpha,\beta)}\Theta^{\alpha-1}(1-\Theta)^{\beta-1}
$$
Here $B(\alpha,\beta) =  \int_0^1 \theta^{\alpha-1}(1-\theta)^{\beta-1}d\theta$ is the normalization constant.
Note that  when $\alpha=\beta=1$, $\Theta$ is uniformly distributed.

Now say that conditioned on $\Theta$, $X_1,\ldots,X_T$ are independent Bernoulli random variables such that $$p(X_i=b|\Theta) = \Theta^b(1-\Theta)^{1-b}$$. Note that $\mathbb{E}[X_t|\Theta]=\Theta$.

Let $M_T = \sum_{t=1}^T X_t$, which is the count of the instances with $X_t=1$.
Then the posterior distribution of $\Theta$ given the samples $X_1,\ldots,X_T$ is given by
$$
p(\Theta| X_1,\ldots,X_T) = \frac{\Theta^{M_T + \alpha-1}(1-\Theta)^{T-M_T+\beta-1}}{B(M_T+\alpha,T-M_T+\beta)}.
$$

We sketched this fact quickly in class. We'll go over it again here for completeness. 

Conditional independence implies that the joint distribution of all of the variables is given by
\begin{align*}
p(\Theta,X_1,\ldots,X_T) &= p(\Theta)\prod_{t=1}^T p(X_t | \Theta)\\
&=\frac{\Theta^{\alpha-1}(1-\Theta)^{\beta-1}}{B(\alpha,\beta)} \prod_{t=1}^T \Theta^{X_t}(1-\Theta)^{1-X_t} \\
&=\frac{\Theta^{M_T + \alpha-1}(1-\Theta)^{T-M_T+\beta-1}}{B(\alpha,\beta)}
\end{align*}

Marginalizing out $\Theta$, along with the definition of the normalizing constant (the Beta function) shows that $p(X_1,\ldots,X_T)=\frac{B(M_T+\alpha,T-M_T+\beta)}{B(\alpha,\beta)}$. The result now follows by Bayes rule.

Returning the case of MDPs with binary rewards, we assume that there are underlying Beta random variables $\Theta(s,a)$ such that $\mathbb{E}[R_{t+1}|S_t=s,A_t=a,\Theta]=\Theta(s,a)$. For simplicity, we can assume that the prior distributions, $p(\Theta(s,a))$, are uniform over $[0,1]$. Then, based on the discussion above, we can compute the posterior distributions of $\Theta(s,a)$ conditioned on the data observed up to the current time by counting the number of times a reward of $1$ was obtained after taking action $a$ from state $s$. 
    
# Dirichlet Distributions and the State Transition Probability.

We must build a model of $p(s'|s,a)$ for all state-action pairs, $(s,a)$. The basic idea is similar to the rewards case. Note that $p(\cdot|s,a)$ is a categorical distribution. The *conjugate prior* of a categorical distribution is the Dirichlet distribution. 

Let's first examine Dirichlet distributions in isolation. A Dirichlet random variable, $Y\in\mathbb{R}^n$ is a vetor in the probability  simplex. In other words, $Y_i\ge 0$ and $\sum_{i=1}^{n} Y_i = 1$.  The density of a Dirichlet random variable is parameterized by $\alpha\in\mathbb{R}^n$ with $\alpha_i >0$, and is given by:
\begin{equation*}
p(Y) = \frac{\prod_{i=1}^n Y_i^{\alpha_i-1}}{B(\alpha)}
\end{equation*}
Here $B(\alpha)$ is the multivariate Beta function, and it is defined by $B(\alpha)=\int_{\Delta^{n-1}} \prod_{i=1}^n y^{\alpha_i-1} dy$, where $\Delta^{n-1}$ is the probability simplex. 

Note that if $\alpha_i=1$ for all $i$, then $Y$ is uniformly distributed.

# Theory Question

Assume that $X_1,\ldots,X_T$ are categorical random variables variables taking values in $\{1,\ldots,n\}$, which are conditionally independent given $Y$. We assume that $p(X_t=i|Y) = Y_i$ and that $Y$ is Dirichlet distributed with parameter $\alpha\in\mathbb{R}^n$.

Let $C_T\in\mathbb{R}^n$ be such that the $i$th coordinate, $C_{T,i}$ is the number of values of $X_t$ such that $X_t=i$. In other words, if $e_1,\ldots,e_n$ are the standard basis vectors of $\mathbb{R}^n$, then $C_T = \sum_{t=1}^T e_{X_t}$.

Show that the posterior distribution, $p(Y|X_1,\ldots,X_T)$, is a Dirichlet distribution with parameter $\alpha+C_T$.

# Back to MDP Modeling

We can let $Y(s,a)$ be Dirichlet distrubtions that give prior models of the state transition distributions $p(\cdot | s,a)$, for  all $(s,a)$. For simplicity, we can assume uniform priors (i.e. $\alpha_i=1$) for all state action pairs.

Then our models of the probability transition distributions can be updated via Bayes rule by counting the number of times that $(S_t,A_t,S_{t+1})=(s,a,s')$ for all $(s,a,s')\in\mathcal{S}\times\mathcal{A}\times\mathcal{S}$ and all transtions observed so far.

So, to be more concrete about the algorithm, at the beginning of each episode, for each pair $(s,a)$, we  draw $\Theta(s,a)\in [0,1]$ to model the expected rewards and $Y(s,a)\in\Delta^{n-1}$ to model state transition probabilities. The parameters for these distributions are calculated from the counts observed from the previous episodes. (In the first episode, these are all drawn uniformly.) 

We construct an MDP from these random variables, and compute  an optimal policy. (In my tests, I found the running a single step of policy iteration works nearly as well as running policy iteration to completion, so this saves a bit of computation.) Then we run a full episode using this policy.

During the episode we update the reward and state transition counts. 

# Coding Question

Code up the Thompson sampling method described above. As with Q-learning, run $10^4$ episodes and plot a running average of the returns of each episode.

Hints and Comments:
- As before, do not include the data $(S_t,A_t,R_{t+1},S_{t+1})$ in the counts if the time limit has been reached at time $t$.
- Do not count the returns of episodes when the time limit has been reached.
- In an episode where the time limit is reached, you CAN include $(S_\tau,A_\tau,R_{\tau+1},S_{\tau+1})$ in the count data if $\tau $ is less than the time limit. (These are just regular trajectory data at that point.)
- Unless you've tuned Q-learning very well, this will probably learn faster and get closer to the true optimal solution. (Though, I did hint at a good set of Q-learning parameters that learns very fast.)

In [ ]:
# Put your Code Here